<a href="https://colab.research.google.com/github/yardsale8/DSCI_210_R_notebooks/blob/main/lecture_6_4_group_and_aggregate.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Grouping and Aggregating in R

In [1]:
library(tidyverse)

── Attaching core tidyverse packages ──────────────────────── tidyverse 2.0.0 ──
✔ dplyr     1.1.3     ✔ readr     2.1.4
✔ forcats   1.0.0     ✔ stringr   1.5.0
✔ ggplot2   3.4.4     ✔ tibble    3.2.1
✔ lubridate 1.9.3     ✔ tidyr     1.3.0
✔ purrr     1.0.2     
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
ℹ Use the conflicted package (<http://conflicted.r-lib.org/>) to force all conflicts to become errors


In [2]:
births <- read.csv('https://github.com/WSU-DataScience/DSCI_210_R_notebooks/raw/main/data/NC_Birth.csv')
head(births)

,Father.Minority,Father.Age,Father.Years.of.Education,Mother.Minority,Mother.Age,Mother.Years.of.Education,Marital.Status,Mother.Smoker,Mother.Avg.Cigs,Mother.Weight.Gain,⋯,Month.of.Birth,Day.of.Birth,Gender,Number.of.Prenatal.Visits,Kotelchuck.Index,Gestational.Age,Birth.Weight.Grams,Low.Birth,Apgar.Score.One.Minute,Apgar.Score.Five.Minutes
,<chr>,<int>,<int>,<chr>,<int>,<int>,<chr>,<chr>,<int>,<int>,⋯,<int>,<int>,<chr>,<chr>,<chr>,<int>,<int>,<chr>,<int>,<int>
1,Nonwhite,50,12,White,24,15,Unmarried,N,0,50,⋯,2,23,Female,10,3_Adequate,38,2866,N,9,9
2,White,19,9,White,18,9,Unmarried,Y,23,35,⋯,1,25,Female,9,3_Adequate,35,3121,N,8,9
3,White,37,17,White,35,17,Married,N,0,24,⋯,1,25,Female,20,4_AdequatePlus,38,2667,N,8,9
4,,NA,NA,Nonwhite,22,14,Unmarried,N,0,33,⋯,11,10,Male,15,4_AdequatePlus,38,2696,N,8,9
5,Nonwhite,39,11,Nonwhite,31,16,Unmarried,N,0,30,⋯,5,18,Female,12,1_Inadequate,38,2979,N,4,9
6,White,20,11,Nonwhite,19,12,Unmarried,N,0,10,⋯,11,6,Male,10,4_AdequatePlus,36,3036,N,9,9


# Outline

* Simple Aggregation
* Grouped Aggregation

# Aggregation

### Simple aggregation

<img src="https://github.com/yardsale8/DSCI_210_R_notebooks/blob/main/img/simple_aggregation.png?raw=1" width=600>

#### Beware of `NA`!

`NA + 1 == NA` in R

In [3]:
(births
 %>% summarize(avg_father_educ = mean(Father.Years.of.Education))
 )

avg_father_educ
<dbl>
NA


#### Option 1 -- Filter out incomplete rows

In [4]:
(births
 %>% filter(complete.cases(Father.Years.of.Education))
 %>% summarize(avg_father_educ = mean(Father.Years.of.Education))
 )

avg_father_educ
<dbl>
12.79394


#### Option 2 -- set `na.rm=TRUE`

In [5]:
(births
 %>% summarize(avg_father_educ = mean(Father.Years.of.Education, na.rm = TRUE))
 )

avg_father_educ
<dbl>
12.79394


### Grouped aggregation

<img src="https://github.com/yardsale8/DSCI_210_R_notebooks/blob/main/img/group_and_aggregate.png?raw=1" width=600>

#### One grouping variable

Using `group_by`

In [10]:
(births
 %>% group_by(Gender)
 %>% summarize(avg_wt = mean(Birth.Weight.Grams,
                             na.rm = TRUE)
              )
 )

Gender,avg_wt
<chr>,<dbl>
Female,3203.117
Male,3279.816


Using `.by` argument in `summarise`

In [11]:
(births
 %>% summarize(avg_wt = mean(Birth.Weight.Grams,
                             na.rm = TRUE),
              .by = Gender)


 )

Gender,avg_wt
<chr>,<dbl>
Female,3203.117
Male,3279.816


#### Two grouping variables

Using `group_by` in the pipe

In [12]:
(births
 %>% group_by(Gender, Marital.Status)
 %>% summarize(avg_wt = mean(Birth.Weight.Grams,
                             na.rm = TRUE)
              )
 )

`summarise()` has grouped output by 'Gender'. You can override using the
`.groups` argument.


Gender,Marital.Status,avg_wt
<chr>,<chr>,<dbl>
Female,Married,3249.788
Female,Unmarried,3137.405
Male,Married,3321.245
Male,Unmarried,3219.448


Using `.by` argument in `summarise`

In [13]:
(births
 %>% summarize(avg_wt = mean(Birth.Weight.Grams,
                             na.rm = TRUE),
               .by = c(Gender, Marital.Status)
              )
 )

Gender,Marital.Status,avg_wt
<chr>,<chr>,<dbl>
Female,Unmarried,3137.405
Female,Married,3249.788
Male,Unmarried,3219.448
Male,Married,3321.245


## <font color="red"> Exercise 6.3.1 </font>

Use `group_by` and/or `summarise` to perform each of the following tasks.

**Tasks:**

1. Compute the overall average (`mean`) and standard deviation(`sd`) of the birth weights (`Birth.Weight.Grams`)
1. Compute the overall average (`mean`) and standard deviation(`sd`) of the birth weights (`Birth.Weight.Grams`) for each category of mother's smoking status (`Mother.Smoker`)

In [ ]:
# Your code here

## Working with a Categorical Response

Similar to the deficiencies we pointed out with JMP's **Table > Summary**,
`dplyr` makes us work when creating two way summary tables for two categorical variables.

### Getting category counts

It is relatively easy to get counts for a single or combination of categorical variables using the `count` function in the pipe. Note that this function performs a grouped aggregation.

In [16]:
(births
 %>% count(Low.Birth, Mother.Smoker)
 )

Low.Birth,Mother.Smoker,n
<chr>,<chr>,<int>
N,,7
N,N,1615
N,Y,203
Y,,1
Y,N,146
Y,Y,28


Alternatively, you can use `group_by` and `summarise` along with the `n` summary function, but this is approach results in needlessly complicated code.

In [17]:
(births
 %>% group_by(Low.Birth, Mother.Smoker)
 %>% summarize(cnt = n())
 )

`summarise()` has grouped output by 'Low.Birth'. You can override using the
`.groups` argument.


Low.Birth,Mother.Smoker,cnt
<chr>,<chr>,<int>
N,,7
N,N,1615
N,Y,203
Y,,1
Y,N,146
Y,Y,28


### Creating "Row percents" in R

Suppose we want the percent of low birth *within* for each smoking status.  We can do this by using `mutate` with the `.by` argument to compute sums within each group.

In [49]:
(births
 %>% count(Mother.Smoker, Low.Birth)
 %>% mutate(cnt_within_smoker = sum(n),
            .by = Mother.Smoker)
 %>% mutate(percent_within_smoker = n/cnt_within_smoker*100)
 )

Mother.Smoker,Low.Birth,n,cnt_within_smoker,percent_within_smoker
<chr>,<chr>,<int>,<int>,<dbl>
,N,7,8,87.500000
,Y,1,8,12.500000
N,N,1615,1761,91.709256
N,Y,146,1761,8.290744
Y,N,203,231,87.878788
Y,Y,28,231,12.121212


## <font color="red"> Exercise 6.3.2 </font>

Use `count` and `mutate` with `.by` to perform each of the following tasks.

**Tasks:**

1. Use `count` to compute the counts for each combination of [Kotelchuck index](https://www-doh.state.nj.us/doh-shad/query/Kotelchuck.html)(`Kotelchuck.Index`) and low birth weight (`Low.Birth`).
2. Compute the counts and row percents when the explanatory variable is the Kotelchuck index (`Kotelchuck.Index`) and the response variable is whether or not there was a low birth weight (`Low.Birth`).  <font size=1>In other words, we want to compute the counts and percents <i>within<i> the Kotelcheck index</font>

In [ ]:
# Your code here

# Translating JMP to `dplyr/tidyr`

### Simple Aggregations

```{r}
surveys %>%
  summarize(
    avg_wgt = mean(weight,
                   na.rm = TRUE))
```

<img width="850" src="https://raw.githubusercontent.com/WSU-DataScience/DSCI_210_R_notebooks/main/img/simple_aggregation_1.png">

<img width="850" src="https://raw.githubusercontent.com/WSU-DataScience/DSCI_210_R_notebooks/main/img/simple_aggregation_2.png">

### Group and Aggregate

```{r}
grp_agg <-
  surveys %>%
    group_by(taxa) %>%
    summarize(avg = mean(weight, na.rm = TRUE))
grp_agg
```

<img width="850" src="https://raw.githubusercontent.com/WSU-DataScience/DSCI_210_R_notebooks/main/img/group_and_aggregate_0.png">

<img width="850" src="https://raw.githubusercontent.com/WSU-DataScience/DSCI_210_R_notebooks/main/img/group_and_aggregate_1.png">

<img width="850" src="https://raw.githubusercontent.com/WSU-DataScience/DSCI_210_R_notebooks/main/img/group_and_aggregate_2.png">

### Multiple Group Variables

<img width="850" src="https://raw.githubusercontent.com/WSU-DataScience/DSCI_210_R_notebooks/main/img/grp2_agg.PNG">